In [1]:
import pandas as pd
import numpy as np
import os

# 1. 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 결과 데이터 로드
# Exp 26: m1+m7 기반의 안정적인 SOTA
exp26_file = f'{SUB_PATH}26_m1_m7_Rank_Ensemble_0.77364.csv'
# Exp 40: m7에 5% PL을 적용한 새로운 로컬 SOTA
exp40_file = f'{SUB_PATH}40_Pure_PL_5pct_AUC_0.77376.csv'

if not os.path.exists(exp26_file) or not os.path.exists(exp40_file):
    print('파일을 찾을 수 없습니다. 경로와 파일명을 확인해주세요.')
else:
    exp26 = pd.read_csv(exp26_file)
    exp40 = pd.read_csv(exp40_file)

    print(f'Blending Exp 26 and Exp 40 (Ratio 6:4)...')

    # 3. Rank Averaging (0~1 Scaling)
    rank26 = exp26['voted'].rank(pct=True)
    rank40 = exp40['voted'].rank(pct=True)

    # 4. 6:4 가중치 결합
    # 안정성(26)에 60%, 실전 패턴(40)에 40%를 배분합니다.
    final_rank = (rank26 * 0.6) + (rank40 * 0.4)

    # 5. 최종 결과 생성
    submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

    # Exp 26의 값 분포를 활용해 순위 재배치
    submission['voted'] = exp26['voted'].sort_values().values[
        (final_rank * (len(final_rank) - 1)).astype(int)
    ]

    # 저장
    output_name = f'{SUB_PATH}41_Master_Ensemble_64.csv'
    submission.to_csv(output_name, index=False)

    print('-' * 50)
    print(f'Exp 41: Master Ensemble (6:4) Created!')
    print(f'Output: {output_name}')
    print('-' * 50)

Blending Exp 26 and Exp 40 (Ratio 6:4)...
--------------------------------------------------
Exp 41: Master Ensemble (6:4) Created!
Output: ./submissions/41_Master_Ensemble_64.csv
--------------------------------------------------
